In [1]:
from skimage import io
from scipy import misc
import os
import openface
import sys
import cv2
import dlib
import tensorflow as tf
from tensorflow import keras
import numpy as np
import imutils
import glob
import facenet
import sklearn 
path_5 = os.path.join("images")
path_1 = os.path.join("images", "face_1.jpeg")
path_2 = os.path.join("models","family_1.jpg")
path_3 = os.path.join("models","dlib_face_recognition_resnet_model_v1.dat")
path_4 = os.path.join("models","shape_predictor_68_face_landmarks.dat")
path_6 = os.path.join("images","5_images","train")
path_7 = os.path.join("images","5_images","val")
path_8 = os.path.join("images","5_images","new_train")

print(cv2.__version__,dlib.__version__,keras.__version__,tf.__version__)
print(path_6,path_3)

/home/ricardo/Personal/data_science/data_science/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ricardo/Personal/data_science/data_science/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ricardo/Personal/data_science/data_science/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/hom

4.1.0 19.17.0 2.1.4-tf 1.7.0
images/5_images/train models/dlib_face_recognition_resnet_model_v1.dat


In [2]:
predictor_path = path_4
face_rec_model_path = path_3

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)
win = dlib.image_window()

final_array_embeddings=[]
final_array_data=[]
final_array_label=[]

for ft1 in glob.glob(os.path.join(path_6, "*")):
    temp_array_data=[]
    temp_array_label=[]
    temp_array_embeddings=[]
    for ft2 in glob.glob(os.path.join(ft1,"*.jpg")):
        #print("Processing file: {}".format(ft2.rsplit('/')[-2]+"/"+ft2.rsplit('/')[-1]))
        img = dlib.load_rgb_image(ft2)
        #print(img.shape,type(img))
        #------------------
        temp_array_data.append(img)
        temp_array_label.append(ft1.rpartition('/')[-1])
        #------------------
        #print(ft1.rpartition('/')[-1],type(ft1.rpartition('/')[-1]))
        #print(type(temp_array_data[0]),temp_array_label[0])
        
        win.clear_overlay()
        win.set_image(img)
        dets = detector(img, 1)
        #print("Number of faces detected: {}".format(len(dets)))
        
        for k,d in enumerate(dets):
            print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
            k+1, d.left(), d.top(), d.right(), d.bottom()))
            # Get the landmarks/parts for the face in box d.
            shape = sp(img, d)
            # Draw the face landmarks on the screen so we can see what face is currently being processed.
            win.clear_overlay()
            win.add_overlay(d)
            win.add_overlay(shape)
            #win.add_overlay()
            #input("Press Enter to continue...")
            face_chip = dlib.get_face_chip(img, shape)        
            #cv2.imwrite("aligned_face_n_{}.jpg".format(k), face_chip)
            #win.set_image(face_chip)
            #win.clear_overlay()
            #input("Press Enter to continue...")
            face_descriptor_from_prealigned_image = facerec.compute_face_descriptor(face_chip)
            temp_array_embeddings.append(face_descriptor_from_prealigned_image)
            cv2.waitKey(0)
        
    final_array_data.extend(temp_array_data)
    final_array_label.extend(temp_array_label)
    final_array_embeddings.extend(temp_array_embeddings)
    
final_array_label_np = np.array(final_array_label)
final_array_embeddings_np =np.array(final_array_embeddings)

Detection 1: Left: 46 Top: 81 Right: 201 Bottom: 236
Detection 1: Left: 12 Top: 81 Right: 167 Bottom: 236
Detection 1: Left: 26 Top: 46 Right: 116 Bottom: 136
Detection 1: Left: 121 Top: 32 Right: 183 Bottom: 94
Detection 1: Left: 53 Top: 96 Right: 182 Bottom: 225
Detection 1: Left: 91 Top: 68 Right: 199 Bottom: 175
Detection 1: Left: 91 Top: 68 Right: 199 Bottom: 175
Detection 1: Left: 52 Top: 32 Right: 114 Bottom: 94
Detection 1: Left: 79 Top: 116 Right: 187 Bottom: 223
Detection 1: Left: 46 Top: 76 Right: 135 Bottom: 166
Detection 1: Left: 101 Top: 38 Right: 152 Bottom: 90
Detection 1: Left: 115 Top: 116 Right: 223 Bottom: 223
Detection 1: Left: 142 Top: 46 Right: 204 Bottom: 108
Detection 1: Left: 73 Top: 59 Right: 135 Bottom: 122
Detection 1: Left: 46 Top: 98 Right: 201 Bottom: 253
Detection 1: Left: 107 Top: 39 Right: 170 Bottom: 101
Detection 1: Left: 52 Top: 39 Right: 114 Bottom: 101
Detection 1: Left: 38 Top: 96 Right: 167 Bottom: 225
Detection 1: Left: 56 Top: 98 Right: 242 B

In [2]:
import dlib
import numpy as np

class photo_embeddings:
    ## this class is programmed the path_to_photos variable contains another carpets
    ## where the carpets names are the labels of the images inside them
    #
    #no class attribute by now
    #
    def __init__(self,path_to_photos,predictor_path,face_rec_model_path):
        self.path_to_photos=path_to_photos
        self.predictor_path=predictor_path
        self.face_rec_model_path=face_rec_model_path
        self.detector = dlib.get_frontal_face_detector()
        self.sp = dlib.shape_predictor(self.predictor_path)
        self.facerec = dlib.face_recognition_model_v1(self.face_rec_model_path)
        
    def get_embeddings(self,):
        
        win = dlib.image_window()
        final_array_embeddings=[]
        final_array_label=[]

        for ft1 in glob.glob(os.path.join(self.path_to_photos, "*")):

            temp_array_label=[]
            temp_array_embeddings=[]
            
            for ft2 in glob.glob(os.path.join(ft1,"*.jpg")):
                
                img = dlib.load_rgb_image(ft2)
                
                dets = self.detector(img, 1)
                win.clear_overlay()
                win.set_image(img)
                
                for k,d in enumerate(dets):
                    print(str(ft2)+','+str(k)+'face')
                    shape = self.sp(img, d)
                    win.clear_overlay()
                    win.add_overlay(d)
                    win.add_overlay(shape)

                    face_chip = dlib.get_face_chip(img, shape)        

                    face_descriptor_from_prealigned_image = self.facerec.compute_face_descriptor(face_chip)
                    temp_array_embeddings.append(face_descriptor_from_prealigned_image)
                    temp_array_label.append(ft1.rpartition('/')[-1])

            final_array_label.extend(temp_array_label)
            final_array_embeddings.extend(temp_array_embeddings)
        
        final_array_label_np = np.array(final_array_label)
        final_array_embeddings_np =np.array(final_array_embeddings)
        
        return final_array_embeddings_np,final_array_label_np
    
    def get_embedding(self,path_to_photo):
        #
        #return the embedding of just a one photo
        #
        #win = dlib.image_window()
        
        temp_array_label=[]
        temp_array_embeddings=[]
        img = dlib.load_rgb_image(path_to_photo)
                
        dets = self.detector(img, 1)
        #win.clear_overlay()
        #win.set_image(img)

        for k,d in enumerate(dets):
            shape = self.sp(img, d)
            #win.clear_overlay()
            #win.add_overlay(d)
            #win.add_overlay(shape)

            face_chip = dlib.get_face_chip(img, shape)        

            face_descriptor_from_prealigned_image = self.facerec.compute_face_descriptor(face_chip)
            temp_array_embeddings.append(face_descriptor_from_prealigned_image)
            temp_array_label.append(path_to_photo.rpartition('/')[-1])
        
        #temp_array_label_np = np.array(temp_array_label)
        #temp_array_embeddings_np =np.array(temp_array_embeddings)
        
        return np.array(temp_array_embeddings),np.array(temp_array_label)
        
    def get_prediction(self,object_model_trained,X_data):
        #
        #return the prediction of a vector of 128 D
        #
        y_pred = object_model_trained.predict(X_data)
        self.y_pred=y_pred
        return y_pred
    
    #def put_label(self,label):
        #
        #this method needs the y_pred result of the get_prediction method to work
        #
        #(a,b,c,d)=__rect_to_bb(rect)
        #return y_label
    
    def __rect_to_bb(self,rect):
        # take a bounding predicted by dlib and convert it
        # to the format (x, y, w, h) as we would normally do
        # with OpenCV
        x = rect.left()
        y = rect.top()
        w = rect.right() - x
        h = rect.bottom() - y

        # return a tuple of (x, y, w, h)
        return (x, y, w, h)
        
        
        

In [3]:
###path_7 = os.path.join("images","5_images","val")
embeddings_obj_new = photo_embeddings(path_7,path_4,path_3)


In [4]:
X_data_new,y_labels_new = embeddings_obj_new.get_embeddings()

images/5_images/val/jerry_seinfeld/httpafilesbiographycomimageuploadcfillcssrgbdprgfacehqwMTIwNjANjMMDINzIxNjcjpg.jpg,0face
images/5_images/val/jerry_seinfeld/httpcdnssninsidercomwpcontentuploadsjerryseinfeldxjpg.jpg,0face
images/5_images/val/jerry_seinfeld/httpaurorasblogcomwpcontentuploadsjerryseinfeldpublicityshotjpg.jpg,0face
images/5_images/val/jerry_seinfeld/httpcdncdnjustjaredcomwpcontentuploadsheadlinesjerryseinfeldmakesbrianwilliamsjokejpg.jpg,0face
images/5_images/val/jerry_seinfeld/httpblognjcomentertainmentimpactcelebritiesmediumjerrybjpg.jpg,0face
images/5_images/val/madonna/httpcdncdnjustjaredcomwpcontentuploadsheadlinesmadonnatalksparisattackstearsjpg.jpg,0face
images/5_images/val/madonna/httpecximagesamazoncomimagesIfmaBKWLACULSRjpg.jpg,0face
images/5_images/val/madonna/httpassetsrollingstonecomassetsarticlemadonnadavidbowiechangedthecourseofmylifeforeversmallsquarexmadonnabowiejpg.jpg,0face
images/5_images/val/madonna/httpassetsrollingstonecomassetsimagesalbumreviewaff

In [5]:
from sklearn.svm import SVC

svm_test_new = SVC(gamma='auto',kernel='sigmoid',decision_function_shape='ovo',probability=False)
svm_test_new.fit(X_data_new,y_labels_new)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [6]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors.nca import NeighborhoodComponentsAnalysis
from sklearn.neighbors import KNeighborsClassifier

nca = NeighborhoodComponentsAnalysis(random_state=42)
nca.fit(X_data_new, y_labels_new)
nca.transform

knn_model_test = KNeighborsClassifier()
knn_model_test.fit(nca.transform(X_data_new),y_labels_new)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [7]:
path_9=os.path.join("images","5_images","val2")
#sklearn.metrics.accuracy_score(y_labels, y_pred)
#y_pred
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)


predictor_path = path_4
face_rec_model_path = path_3

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

#for ft1 in glob.glob(os.path.join(path_7, "*")):

    #for ft2 in glob.glob(os.path.join(ft1,"*.jpg")):
for ft2 in glob.glob(os.path.join(path_9,"*.jpg")):

    #image=dlib.load_rgb_image(ft2)
    image = cv2.imread(ft2)
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    rects = detector(gray, 1)

    #fname = ft1.rpartition('/')[-1]

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        #print("{} : faces".format(i+1))
        (x, y, w, h) = rect_to_bb(rect)
        shape = sp(gray, rect)
        #win.clear_overlay()
        #win.add_overlay(rect)
        #win.add_overlay(shape)
        face_chip = dlib.get_face_chip(image, shape)        
        face_descriptor_from_prealigned_image = facerec.compute_face_descriptor(face_chip)
        #predicted_label=svm_test.predict(np.array(face_descriptor_from_prealigned_image).reshape(1,-1))

        ### HERE, we are knowing the confidence rating for asigning a good label
        nca_embeddings = nca.transform(np.array(face_descriptor_from_prealigned_image).reshape(1,-1))
        confidence_rate = knn_model_test.predict_proba(nca_embeddings.reshape(1,-1))
        confidence_rate= np.amax(confidence_rate[0])*100
        #print(confidence_rate[0][0])
        label='unknown'
        if(confidence_rate>=65):
            predicted_label=knn_model_test.predict(nca_embeddings.reshape(1,-1))
            label=str(predicted_label[0])

        clone = image.copy()
        cv2.rectangle(clone, (x, y),(x + w, y + h), (0, 255, 0), 3)
        startX = x
        startY = y - 15 if y - 15 > 15 else y + 15
        cv2.putText(clone, label + ' '+str(confidence_rate)+'%', (startX, startY),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        roi = image[y:y + h, x:x + w]
        #cv2.imshow("Image", clone)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        cv2.imshow("Image", clone)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    
            
cv2.destroyAllWindows()

In [33]:
path_15=os.path.join("images","5_images","new_train","pablo","photoP1.jpg")
path_16=os.path.join("images","local_images","train_5","percy","PE5.jpg")
X_data_new,y_labels_new = embeddings_obj_new.get_embedding(path_16)



In [34]:
svm_test_new.predict(X_data_new)

array(['mindy_kaling'], dtype='<U14')

In [ ]:
path_16 = os.path.join("images","local_images","train_5")

In [35]:
knn_model_test.predict(nca.transform(X_data_new))

array(['elton_john'], dtype='<U14')

# TRAINING ON PHOTOS

In [8]:
##train directory
path_11=os.path.join("images","local_images","train_5")
##test_directory
path_12=os.path.join("images","local_images","photos_test")

In [9]:
embeddings_obj_local = photo_embeddings(path_11,path_4,path_3)

In [10]:
X_data_new,y_labels_new = embeddings_obj_local.get_embeddings()

images/local_images/train_5/pablo/photoP1.jpg,0face
images/local_images/train_5/pablo/P7.jpg,0face
images/local_images/train_5/pablo/photoP2.jpg,0face
images/local_images/train_5/pablo/photoP5.jpg,0face
images/local_images/train_5/donny/D9.jpg,0face
images/local_images/train_5/donny/D1.jpg,0face
images/local_images/train_5/donny/1918714_214177568292_679051_n.jpg,0face
images/local_images/train_5/donny/D5.jpg,0face
images/local_images/train_5/donny/D10.jpg,0face
images/local_images/train_5/flor/F1.jpg,0face
images/local_images/train_5/flor/F2.jpg,0face
images/local_images/train_5/flor/f8.jpg,0face
images/local_images/train_5/flor/F7.jpg,0face
images/local_images/train_5/percy/photoPE2.jpg,0face
images/local_images/train_5/percy/PE4.jpg,0face
images/local_images/train_5/percy/PE5.jpg,0face


In [41]:
X_data_new.shape
print(y_labels_new)

['pablo' 'pablo' 'pablo' 'pablo' 'donny' 'donny' 'donny' 'donny' 'donny'
 'flor' 'flor' 'flor' 'flor' 'percy' 'percy' 'percy']


In [11]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors.nca import NeighborhoodComponentsAnalysis
from sklearn.neighbors import KNeighborsClassifier

nca = NeighborhoodComponentsAnalysis(random_state=42)
nca.fit(X_data_new, y_labels_new)
nca.transform

knn_model_test = KNeighborsClassifier(n_neighbors=4)
knn_model_test.fit(nca.transform(X_data_new),y_labels_new)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [12]:
#path_9=os.path.join("images","5_images","val2")
#sklearn.metrics.accuracy_score(y_labels, y_pred)
#y_pred
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)


predictor_path = path_4
face_rec_model_path = path_3

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

#for ft1 in glob.glob(os.path.join(path_7, "*")):

    #for ft2 in glob.glob(os.path.join(ft1,"*.jpg")):
for ft2 in glob.glob(os.path.join(path_12,"*.jpg")):

    #image=dlib.load_rgb_image(ft2)
    image = cv2.imread(ft2)
    image = imutils.resize(image, width=500,height=500)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    rects = detector(gray, 1)

    #fname = ft1.rpartition('/')[-1]

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        #print("{} : faces".format(i+1))
        (x, y, w, h) = rect_to_bb(rect)
        shape = sp(gray, rect)
        #win.clear_overlay()
        #win.add_overlay(rect)
        #win.add_overlay(shape)
        face_chip = dlib.get_face_chip(image, shape)        
        face_descriptor_from_prealigned_image = facerec.compute_face_descriptor(face_chip)
        #predicted_label=svm_test.predict(np.array(face_descriptor_from_prealigned_image).reshape(1,-1))

        ### HERE, we are knowing the confidence rating for asigning a good label
        nca_embeddings = nca.transform(np.array(face_descriptor_from_prealigned_image).reshape(1,-1))
        confidence_rate = knn_model_test.predict_proba(nca_embeddings.reshape(1,-1))
        confidence_rate= np.amax(confidence_rate[0])*100
        #print(confidence_rate[0][0])
        label='unknown'
        if(confidence_rate>=55):
            predicted_label=knn_model_test.predict(nca_embeddings.reshape(1,-1))
            label=str(predicted_label[0])

        clone = image.copy()
        cv2.rectangle(clone, (x, y),(x + w, y + h), (0, 255, 0), 3)
        startX = x
        startY = y - 15 if y - 15 > 15 else y + 15
        cv2.putText(clone, label + ' '+str(confidence_rate)+'%', (startX, startY),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        roi = image[y:y + h, x:x + w]
        #cv2.imshow("Image", clone)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        cv2.imshow("Image", clone)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    
            
cv2.destroyAllWindows()

In [ ]:
dist = numpy.linalg.norm(a-b)
